# mlp_autoencoder_cifar10_fashionmnist

El script tiene dos objetivos principales claramente diferenciados:

---

1. **Entrenamiento de un MLP para clasificación de imágenes (CIFAR-10)**

  * Construir un perceptrón multicapa (MLP) en Keras para clasificar imágenes del dataset CIFAR-10.

  * Preprocesar los datos (normalización y visualización inicial).

  * Entrenar el modelo, probando diferentes hiperparámetros y arquitecturas (capas y neuronas) para mejorar el rendimiento en el conjunto de validación/test.

  * Medir y reportar métricas clave:

    * Pérdida y exactitud en entrenamiento y validación.

    * Velocidad por época.

  * Graficar la evolución de pérdida y exactitud durante el entrenamiento.

2. **Construcción y análisis de autoencoders (Fashion-MNIST)**

  * Construir autoencoders en Keras con distintas dimensiones del "código" (representación comprimida): 16, 32 y 64.

  * Entrenar los modelos para minimizar la pérdida de reconstrucción.

  * Comparar el desempeño entre entrenamiento y validación/test según el tamaño de la representación.

  * Visualizar resultados:

    * Imágenes originales vs reconstruidas.

    * Evolución de la pérdida por época.

    * Relación entre el tamaño del código y la pérdida promedio de reconstrucción.

---

En resumen, el script busca enseñar y comparar dos aplicaciones diferentes de redes neuronales:

1. Clasificación supervisada (MLP en CIFAR-10).

2. Aprendizaje no supervisado mediante autoencoders (compresión y reconstrucción en Fashion-MNIST).

# Parte 1: MLP con imágenes

•Implemente en Keras un MLP con la cantidad de capas y neuronas que prefiera, que permita hacer clasificación de imágenes.

•Entrene la red en CIFAR10 y reporte valores relevantes: velocidad por época, pérdida entrenamiento, pérdida validación, etc.

•Juegue con los hiperparámetros y estructura de la red hasta obtener el rendimiento más alta en el set de validación/test.

In [ ]:
# librerias clasicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]
plt.rcParams.update({'font.size': 16})

import warnings
warnings.filterwarnings("ignore")

#libreria sklearn
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import datasets, layers, models


### Carga del dataset CIFAR-10

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays,
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

### Construcción del modelo MLP

In [ ]:
input_shape = (32 ,32, 3) # Imagenes de 32 x 32 con 3 canales

In [ ]:
model = models.Sequential()
model.add(layers.InputLayer(input_shape= input_shape))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

model.summary()

### Entrenamiento del Modelo

In [ ]:
train_images.shape

In [ ]:
test_images.shape

from_logits = TRUE --> Internamente usa softmax

In [ ]:
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=128, epochs=50, validation_split=0.1)

### Evaluación

In [ ]:
score = model.evaluate(test_images, test_labels, verbose = 0)
print("Loss en conjunto de test:",round(score[0],3))
print("Accuracy en conjunto de test:", round(score[1],3))

### Rporte de valores relevantes y gráficos

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

**Accuracy**

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='lower right')

# Parte 2: autoencoder

Un autoencoder es una arquitectura de red neuronal que se utiliza para aprender features generativas, es decir, que permitan generar datos.
En particular, un autoencoder utiliza un estructura de cuello de botella para generar un output que sea lo más parecido posible al input.

•Utilizando Keras, construya y entrene autoencoders para alguno de los sets de datos disponibles.

•Haga un análisis de sensibilidad para el código (code en la figura), graficando su tamaño vs la pérdida de reconstrucción promedio. Compare estos valores para los sets de entrenamiento y validación/test.


### Carga de datos

In [ ]:
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

In [ ]:
# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)

    plt.xlabel(class_names[y_train[i]])
plt.show()

### Construcción del modelo

In [ ]:
def build_autoencoder(input_shape, representation_size):
    encoder = models.Sequential()
    encoder.add(layers.InputLayer(input_shape=input_shape))
    encoder.add(layers.Flatten())
    encoder.add(layers.Dense(256, activation='relu'))
    encoder.add(layers.Dense(128, activation='relu'))
    encoder.add(layers.Dense(representation_size, activation='relu'))

    decoder = models.Sequential()
    decoder.add(layers.Dense(128, activation='sigmoid'))
    decoder.add(layers.Dense(256, activation='sigmoid'))
    decoder.add(layers.Dense(np.prod(input_shape), activation='sigmoid'))
    decoder.add(layers.Reshape(input_shape)) # "Unflatten"

    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    model.add(encoder)
    model.add(decoder)

    return model, encoder, decoder

In [ ]:
input_shape = x_train.shape[1:]

## Representación de tamaño 16

In [ ]:
model16, encoder, decoder = build_autoencoder(input_shape, 16)

In [ ]:
model16.summary(expand_nested=True)

In [ ]:
model16.compile(optimizer='adam', loss='binary_crossentropy')

history16 = model16.fit(x_train, x_train, batch_size=128, epochs=50, shuffle=True, validation_split=0.1)

In [ ]:
plt.plot(history16.history['loss'], label='loss')
plt.plot(history16.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='lower right')
plt.show()

In [ ]:
decoded_imgs = model16(x_test[:10])

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i])
    plt.title("Original", fontsize=12)
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i])
    plt.title("Reconstructed", fontsize=12)
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Representación de tamaño 32

In [ ]:
model32, encoder, decoder = build_autoencoder(input_shape, 32)

In [ ]:
model32.summary(expand_nested=True)

In [ ]:
model32.compile(optimizer='adam', loss='binary_crossentropy')

history32 = model32.fit(x_train, x_train, batch_size=128, epochs=50, shuffle=True, validation_split=0.1)

In [ ]:
plt.plot(history32.history['loss'], label='loss')
plt.plot(history32.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='lower right')
plt.show()

In [ ]:
decoded_imgs = model32(x_test[:10])

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i])
    plt.title("Original", fontsize=12)
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i])
    plt.title("Reconstructed", fontsize=12)
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Representación de tamaño 64

In [ ]:
model64, encoder, decoder = build_autoencoder(input_shape, 64)

In [ ]:
model64.summary(expand_nested=True)

In [ ]:
model64.compile(optimizer='adam', loss='binary_crossentropy')

history64 = model64.fit(x_train, x_train, batch_size=128, epochs=50, shuffle=True, validation_split=0.1)

In [ ]:
plt.plot(history64.history['loss'], label='loss')
plt.plot(history64.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='lower right')
plt.show()

In [ ]:
decoded_imgs = model64(x_test[:10])

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i])
    plt.title("Original", fontsize=12)
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i])
    plt.title("Reconstructed", fontsize=12)
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
train_score16 = model16.evaluate(x_train, x_train, verbose = 0)
test_score16 = model16.evaluate(x_test, x_test, verbose = 0)

train_score32 = model32.evaluate(x_train, x_train, verbose = 0)
test_score32 = model32.evaluate(x_test, x_test, verbose = 0)

train_score64 = model64.evaluate(x_train, x_train, verbose = 0)
test_score64 = model64.evaluate(x_test, x_test, verbose = 0)

In [ ]:
sizes = [16, 32, 64]
plt.plot(sizes, [train_score16, train_score32, train_score64], label="Train Mean Loss")
plt.plot(sizes, [test_score16, test_score32, test_score64], label="Test Mean Loss")
plt.xlabel("Representation size")
plt.xticks([16, 32, 64])
plt.legend()